# **重要：CPUのランタイムで起動してください**
　**このプログラムにはGPU上で実行不能な関数が含まれます．**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/sueki_data/20220222_sueki_dataset.zip

# パッケージインストール
* 最後の3D可視化で，matplotlib3.3以降から追加された関数：set_box_aspect（3Dグラフにおけるアスペクト比の調整用関数）を使用します．
* 2022/02/28現在におけるClabのmatplotlibは3.2なのでアップグレードをお願いいたします．
* albumentationsは依存関係チェックを回避するためにアップグレードします．
* **アップグレード後にランタイムを再起動して下さい（ランタイム再起動しないと正常に反映されません）**

In [ ]:
!pip uninstall -q albumentations matplotlib matplotlib-inline matplotlib-venn
!pip install -q albumentations matplotlib matplotlib-inline matplotlib-venn

Proceed (y/n)? y
Proceed (y/n)? y
Proceed (y/n)? y
Proceed (y/n)? y
     |████████████████████████████████| 102 kB 5.3 MB/s 
     |████████████████████████████████| 11.2 MB 35.9 MB/s 
     |████████████████████████████████| 47.7 MB 1.4 MB/s 
     |████████████████████████████████| 895 kB 59.3 MB/s 


In [ ]:
!pip install -q japanize_matplotlib
!pip install -q tensorflow-determinism classification-models-3D

     |████████████████████████████████| 4.1 MB 5.3 MB/s 
     |████████████████████████████████| 46 kB 1.8 MB/s 


# **ランタイム再起動**
* matplotlibのアップグレードを反映させるため
* 再起動しない場合，次のパッケージインポートでエラーが出ます．

# 仮ラベル付きのクラスタリング

In [ ]:
import os
from glob import glob
import re
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random
import gc

from skimage.transform import resize
from matplotlib import pyplot as plt
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
from matplotlib import cm
from keras.utils import np_utils

In [ ]:

def set_seed(seed=200):
    tf.random.set_seed(seed)

    # optional
    # for numpy.random
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)
    # for hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    

set_seed(123)


## 定数定義
　可視化するCNNCluster3Dモデルと定数定義を合わせてください．

In [ ]:
NUM_CLASSES = 8
NUM_CLUSTERS = 8
NUM_PCA_COMPONENTS = 8
BUFFER_SIZE = 1000
BATCH_SIZE = 16 
EPOCHS = 50
CLUSTERING_INTERVAL = 1
DATA_DIR = '/content/data/voxel'
MI_FUTA = 'futa'
IMAGE_SIZE = 64
NUM_CHANNELS = 4

FEATURE_EXTRACTOR_MODEL_PATH = "feature_extractor_20220222-selu-i_{}_cls{}_ep{}.hdf5".format(MI_FUTA,NUM_CLUSTERS,EPOCHS)
CLASSIFIER_MODEL_PATH = "classifier_20220222-selu-i_{}_cls{}_ep{}.hdf5".format(MI_FUTA,NUM_CLUSTERS,EPOCHS)
DATASET_SAVE_PATH = "pseudo_dataset_20220222-selu-i_{}_cls{}_ep{}".format(MI_FUTA,NUM_CLUSTERS,EPOCHS)

DATASAVE_DIR = "{}_20220222-selu-i_deg30_cls{}_ep{}".format(MI_FUTA,NUM_CLUSTERS,EPOCHS)


## モデル定義

In [ ]:
class ConvNet3D(keras.Model):
    def __init__(self):
        super(ConvNet3D, self).__init__()
        self.conv3d_1 = keras.layers.Conv3D(filters=64, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_1')
        self.bn_1 = keras.layers.BatchNormalization()
        self.act_1 = keras.layers.Activation('selu')
        self.maxpool3d_1 = keras.layers.MaxPool3D(pool_size=2)
        
        
        self.conv3d_2 = keras.layers.Conv3D(filters=64, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_2')
        self.bn_2 = keras.layers.BatchNormalization()
        self.act_2 = keras.layers.Activation('selu')
        self.maxpool3d_2 = keras.layers.MaxPool3D(pool_size=2)
        
        
        self.conv3d_3 = keras.layers.Conv3D(filters=128, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_3')
        self.bn_3 = keras.layers.BatchNormalization()
        self.act_3 = keras.layers.Activation('selu')
        self.maxpool3d_3 = keras.layers.MaxPool3D(pool_size=2)
        
        
        self.conv3d_4 = keras.layers.Conv3D(filters=256, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_4')
        self.bn_4 = keras.layers.BatchNormalization()
        self.act_4 = keras.layers.Activation('selu')
        self.maxpool3d_4 = keras.layers.MaxPool3D(pool_size=1)
        

        self.gap_1 = keras.layers.GlobalAveragePooling3D()
        
        self.dense_1 = keras.layers.Dense(128, kernel_initializer='lecun_normal')
        self.dense_2 = keras.layers.Dense(128, kernel_initializer='lecun_normal')

    def call(self, inputs):
        x = self.conv3d_1(inputs)
        x = self.bn_1(x)
        x = self.act_1(x)
        x = self.maxpool3d_1(x)
        
        x = self.conv3d_2(x)
        x = self.bn_2(x)
        x = self.act_2(x)
        x = self.maxpool3d_2(x)
        
        x = self.conv3d_3(x)
        x = self.bn_3(x)
        x = self.act_3(x)
        x = self.maxpool3d_3(x)
        
        x = self.conv3d_4(x)
        x = self.bn_4(x)
        x = self.act_4(x)
        x = self.maxpool3d_4(x)
        
        x = self.gap_1(x)
        x_1 = self.dense_1(x)
        x_2 = self.dense_2(x)
        return [x_1, x_2]

In [ ]:
class Classify3D(tf.keras.Model):
    def __init__(self, feature_extractor, num_of_classes=3):
        super(Classify3D, self).__init__()
        self.feature_extractor = feature_extractor
        self.num_of_classes = num_of_classes
        self.activation_1 = keras.layers.Activation(keras.activations.selu)
        self.activation_2 = keras.layers.Activation(keras.activations.selu)
        self.dense_1 = keras.layers.Dense(self.num_of_classes, activation='softmax')
        self.dense_2 = keras.layers.Dense(self.num_of_classes, activation='softmax')
        
    def call(self, inputs):
        x_1, x_2 = self.feature_extractor(inputs)
        x_1 = self.activation_1(x_1)
        x_1 = self.dense_1(x_1)
        x_2 = self.activation_2(x_2)
        x_2 = self.dense_2(x_2)

        return [x_1, x_2]

In [ ]:
feature_extractor = ConvNet3D()
feature_extractor.build(input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
feature_extractor.summary()

Model: "conv_net3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_1 (Conv3D)           multiple                  6976      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 activation (Activation)     multiple                  0         
                                                                 
 max_pooling3d (MaxPooling3D  multiple                 0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           multiple                  110656    
                                                                 
 batch_normalization_1 (Batc  multiple                 2

In [ ]:
classifier = Classify3D(feature_extractor, num_of_classes=NUM_CLASSES)
classifier.build(input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
classifier.summary()

Model: "classify3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_net3d (ConvNet3D)      multiple                  1291776   
                                                                 
 activation_4 (Activation)   multiple                  0         
                                                                 
 activation_5 (Activation)   multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  1032      
                                                                 
 dense_3 (Dense)             multiple                  1032      
                                                                 
Total params: 1,293,840
Trainable params: 1,292,816
Non-trainable params: 1,024
_________________________________________________________________


## データ準備

In [ ]:
categories1 = ['2-1','2-2','2-3','2-4','2-5','temp1-1','temp1-2','temp1-3']#,'temp1-4','temp1-5']
categories2 = ['前葉','中葉','後葉', 'temp2-1','temp2-2','temp2-3','temp2-4','temp2-5']#,'temp2-6','temp2-7']

In [ ]:
label_dict = {
  'J-3651':['2-3','中葉'],#蓋  ##追加20220217
  'J-13305-5':['2-3','中葉'],#蓋
  'J-6568':['2-3','中葉'],#蓋
  'J-22896-1':['2-4','後葉'],#蓋
  'J-22896-2':['2-4','後葉'],#蓋
  'J-22896-4':['2-4','後葉'],#蓋
  'J-3457':['2-5','後葉'],#蓋
  'J-6569':['2-5','後葉'],#蓋
  'J-14876':['2-5','後葉'],#蓋  ##追加20220222
  'J-20215':['2-5','後葉'],#蓋
  'J-22896-3':['2-5','後葉'],#蓋
  'J-22896-9':['2-5','後葉'],#蓋  ##追加20220217
  'J-23682':['2-5','後葉'],#蓋  ##追加20220222

  'J-6564':['2-3','中葉'],#蓋と身
  'J-6574':['2-3','中葉'],#蓋と身
  'J-13305-2':['2-3','中葉'],#蓋と身
  'J-23050':['2-4','後葉'],#蓋と身  ##20220210追加
  'J-11536':['2-5','後葉'],#蓋と身  ##追加20220222


  'J-142':['2-3','中葉'],#身 ##追加20220214
  'J-3516':['2-3','中葉'],#身  ##追加20220214
  'J-3527':['2-3','中葉'],#身  ##追加20220214
  'J-3583':['2-3','中葉'],#身
  'J-5834':['2-3','中葉'],#身  ##追加20220217
  'J-6565':['2-3','中葉'],#身
  'J-3603':['2-4','後葉'],#身 ##20220208追加
  'J-6571':['2-4','後葉'],#身

  'J-8828-3':['2-4','後葉'],#身
  'J-8828-5':['2-4','後葉'],#身
  'J-22896-15':['2-4','後葉'],#身  ##追加20220217
  'J-3650':['2-5','後葉'],#身  ##追加20220222
  'J-9830':['2-5','後葉'],#身
  'J-22266':['2-5','後葉'],#身
  'J-22896-12':['2-5','後葉'],#身  ##追加20220217
  'J-22896-16':['2-5','後葉'],#身  ##追加20220217
  'J-22896-19':['2-5','後葉'],#身  ##追加20220222
  'J-22896-21':['2-5','後葉'],#身  ##追加20220222
  
  'J-139':['2-1','前葉'],#蓋と身  ##追加20220217
  'J-11563':['2-1','前葉'],#蓋と身  ##追加20220217
  
  'J-7598':['2-1','前葉'],#蓋と身  ##20220210追加
  'J-3961':['2-1','前葉'],#身  ##追加20220214
  'J-20761':['2-1','前葉'],#身  ##追加20220214


  'J-147':['2-2','前葉'],#蓋 ##追加20220214
  'J-3522':['2-2','前葉'],#蓋

  'J-3920':['2-2','前葉'],#身  ##追加20220214
  'J-7606':['2-2','前葉'],#身  ##追加20220214
  'J-22802':['2-2','前葉'],#身


  'J-6572':['2-3','中葉'],#蓋
  'J-13305-3':['2-3','中葉'],#蓋
  'J-13305-6':['2-3','中葉'],#蓋
  'J-6570':['2-4','後葉'],#蓋
  'J-23049':['2-4','後葉'],#蓋  ##20220210追加
  'J-22896-8':['2-4','後葉'], #蓋

  'J-3667':['2-5','後葉'],#蓋
  'J-22896-7':['2-5','後葉'],#蓋
  'J-22896-10':['2-5','後葉'],#蓋  ##追加20220217

  
  'J-6573':['2-3','中葉'],#蓋と身
  'J-13305-1':['2-5','後葉'],#蓋と身
  'J-6566':['2-3','中葉'],#身
  'J-3648':['2-4','後葉'],#身
  'J-8828-6':['2-4','後葉'],#身
  'J-33902':['2-4','後葉'],#身 ##20220208追加
  'J-3619':['2-5','後葉'],#身 ##20220208追加
  'J-8828-8':['2-5','後葉'],#身
  'J-22896-13':['2-5','後葉'],#身  ##追加20220217
  'J-22896-22':['2-5','後葉'],#身  ##追加20220222


  'J-20295':['2-1','前葉'],#蓋  ##追加20220217
  'J-4120':['2-1','前葉'],#蓋と身 ##追加20220214


  'J-3589':['2-2','前葉'],#蓋  ##追加20220214
  'J-3634':['2-2','前葉'],#蓋と身  ##20220208追加  
  'J-3959':['2-2','前葉'],#身  ##追加20220214
  

  'J-3584':['2-3','中葉'],#蓋
  'J-13305-4':['2-3','中葉'],#蓋
  'J-22896-5':['2-4','後葉'],#蓋
  'J-22262':['2-5','後葉'],#蓋
  'J-22896-6':['2-5','後葉'],#蓋
  'J-22896-11':['2-5','後葉'],#蓋  ##追加20220217

  'J-134':['2-3','中葉'],#蓋と身 ##20220210追加（身が20220222で追加）
  'J-6575':['2-3','中葉'],#蓋と身

  'J-4121':['2-3','中葉'],#身 ##追加20220214
  'J-6576':['2-3','中葉'],#身
  'J-8828-7':['2-3','中葉'],#身
  'J-3453':['2-4','後葉'],#身
  'J-7337':['2-4','後葉'],#身
  'J-8828-4':['2-4','後葉'],#身
  'J-22896-17':['2-4','後葉'],#身  ##追加20220222
  'J-3510':['2-5','後葉'],#身  ##追加20220222
  'J-33900':['2-5','後葉'],#身
  'J-22896-14':['2-5','後葉'],#身  ##追加20220217

  'J-20297':['2-1','前葉'],#蓋  ##追加20220217
  'J-22806':['2-1','前葉'],#身

  'J-4081':['2-2','前葉'],#蓋  ##追加20220214
  'J-7607':['2-2','前葉'],#蓋と身  ##20220210追加
  'J-3682':['2-2','前葉'],#身  ##追加20220214

}


# Grad-CAM

## CNNClusterで作成したモデルとデータセットをコピー

In [ ]:
# ドライブへ退避させたモデルとデータセットをコピー
#!cp -r /content/drive/MyDrive/sueki_data/{DATASAVE_DIR}/{FEATURE_EXTRACTOR_MODEL_PATH} /content/drive/MyDrive/sueki_data/{DATASAVE_DIR}/{CLASSIFIER_MODEL_PATH} /content/drive/MyDrive/sueki_data/{DATASAVE_DIR}/{DATASET_SAVE_PATH} ./

In [ ]:
# ドライブへ退避させたモデルとデータセットをコピー
!cp -r /content/drive/MyDrive/sueki_data/{DATASAVE_DIR}/ ./

## Grad-CAM用のモデル定義
  - 上記のクラスタリングのモデル定義をまとめて1つにしたもの<br>
  - Classify3Dクラスで定義したモデルからConvNet3Dクラスのconv3d_4レイヤを直接get_layerできないため1つのクラスで再定義<br>
  
  

In [ ]:
class ConvNet3D_grad(keras.Model):
    def __init__(self,num_of_classes=8):
        super(ConvNet3D_grad, self).__init__()
        self.conv3d_1 = keras.layers.Conv3D(filters=64, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_1')
        self.bn_1 = keras.layers.BatchNormalization()
        self.act_1 = keras.layers.Activation('selu')
        self.maxpool3d_1 = keras.layers.MaxPool3D(pool_size=2)
        
        
        self.conv3d_2 = keras.layers.Conv3D(filters=64, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_2')
        self.bn_2 = keras.layers.BatchNormalization()
        self.act_2 = keras.layers.Activation('selu')
        self.maxpool3d_2 = keras.layers.MaxPool3D(pool_size=2)
        
        
        self.conv3d_3 = keras.layers.Conv3D(filters=128, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_3')
        self.bn_3 = keras.layers.BatchNormalization()
        self.act_3 = keras.layers.Activation('selu')
        self.maxpool3d_3 = keras.layers.MaxPool3D(pool_size=2)
        
        
        self.conv3d_4 = keras.layers.Conv3D(filters=256, kernel_initializer='lecun_normal', kernel_size=3, activation='linear', name='conv3d_4')
        self.bn_4 = keras.layers.BatchNormalization()
        self.act_4 = keras.layers.Activation('selu')
        self.maxpool3d_4 = keras.layers.MaxPool3D(pool_size=1)
        

        self.gap_1 = keras.layers.GlobalAveragePooling3D()
        
               
        self.dense_1_1 = keras.layers.Dense(128, kernel_initializer='lecun_normal')
        self.dense_1_2 = keras.layers.Dense(128, kernel_initializer='lecun_normal')

        self.num_of_classes = num_of_classes

        self.activation_2_1 = keras.layers.Activation(keras.activations.selu)
        self.activation_2_2 = keras.layers.Activation(keras.activations.selu)
        self.dense_2_1 = keras.layers.Dense(self.num_of_classes, activation='softmax')
        self.dense_2_2 = keras.layers.Dense(self.num_of_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv3d_1(inputs)
        x = self.bn_1(x)
        x = self.act_1(x)
        x = self.maxpool3d_1(x)
        
        x = self.conv3d_2(x)
        x = self.bn_2(x)
        x = self.act_2(x)
        x = self.maxpool3d_2(x)
        
        x = self.conv3d_3(x)
        x = self.bn_3(x)
        x = self.act_3(x)
        x = self.maxpool3d_3(x)
        
        x = self.conv3d_4(x)
        x = self.bn_4(x)
        x_g = self.act_4(x)
        x_g = self.maxpool3d_4(x_g)
        
        x_g = self.gap_1(x_g)
        x_1 = self.dense_1_1(x_g)
        x_2 = self.dense_1_2(x_g)

        x_1 = self.activation_2_1(x_1)
        x_1 = self.dense_2_1(x_1)

        x_2 = self.activation_2_2(x_2)
        x_2 = self.dense_2_2(x_2)


        return [x, [x_1, x_2]]

In [ ]:
gc.collect()

99

### CNNCluster3Dモデルの重みをLoadする

In [ ]:
temp_model_1 = ConvNet3D()
temp_model_1.build(input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
temp_model_1.load_weights(DATASAVE_DIR+'/'+FEATURE_EXTRACTOR_MODEL_PATH)
temp_model_2 = Classify3D(temp_model_1, num_of_classes=NUM_CLASSES)
temp_model_2.build(input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
temp_model_2.load_weights(DATASAVE_DIR+'/'+CLASSIFIER_MODEL_PATH)

In [ ]:
grad_model = ConvNet3D_grad()
grad_model.build(input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))

#既存モデルをLoad
grad_model.set_weights(temp_model_2.weights)

# Grad-CAM用dataframe
　CNNCluster3Dで作成した疑似ラベル付きのdatasetと，クラスタ結果のnpyファイルを読み込みます．

In [ ]:
pseudo_dataset = tf.data.experimental.load(DATASAVE_DIR+'/'+DATASET_SAVE_PATH)
km_predictions1 = np.load(DATASAVE_DIR+"/km_predictions1.npy")
km_predictions2 = np.load(DATASAVE_DIR+"/km_predictions2.npy")

In [ ]:
num = []
futami = []
name = []
true_type = []
true_age = []
pseudo_type = []
pseudo_age = []
cluster_type = []
cluster_age = []
voxel_path = []

for i, d in enumerate(pseudo_dataset):
  voxel_path.append(d[0].numpy().decode())
  futami.append(os.path.splitext(os.path.basename(d[0].numpy().decode()))[0].split('_',4)[0])
  name.append(os.path.splitext(os.path.basename(d[0].numpy().decode()))[0].split('_',4)[1])
  true_type.append(categories1[int(os.path.splitext(os.path.basename(d[0].numpy().decode()))[0].split('_',4)[2])])
  true_age.append(categories2[int(os.path.splitext(os.path.basename(d[0].numpy().decode()))[0].split('_',4)[3])])
  pseudo_type.append(d[1].numpy()[0])
  pseudo_age.append(d[1].numpy()[1])
  num.append(os.path.splitext(os.path.basename(d[0].numpy().decode()))[0].split('_',6)[5])

for k1, k2 in zip(km_predictions1,km_predictions2):
  cluster_type.append(k1)
  cluster_age.append(k2)
  
pseudo_df = pd.DataFrame(list(zip(futami, name, voxel_path, num, true_type, true_age, pseudo_type, pseudo_age, cluster_type, cluster_age)), columns = \
                  ['蓋・身','列品番号','voxel_path','回転角度','正解ラベル-型式','正解ラベル-時代','疑似ラベル-型式','疑似ラベル-時代','クラスタ-型式','クラスタ-時代'])

In [ ]:
pseudo_df.to_csv(DATASET_SAVE_PATH+".csv")

In [ ]:
pseudo_df

,蓋・身,列品番号,voxel_path,回転角度,正解ラベル-型式,正解ラベル-時代,疑似ラベル-型式,疑似ラベル-時代,クラスタ-型式,クラスタ-時代
0,futa,J-11536,/content/data/voxel/futa_J-11536_4_2_64_000.npy,000,2-5,後葉,2,1,1,5
1,futa,J-11536,/content/data/voxel/futa_J-11536_4_2_64_030.npy,030,2-5,後葉,2,1,1,5
2,futa,J-11536,/content/data/voxel/futa_J-11536_4_2_64_060.npy,060,2-5,後葉,2,1,1,5
3,futa,J-11536,/content/data/voxel/futa_J-11536_4_2_64_090.npy,090,2-5,後葉,2,1,1,5
4,futa,J-11536,/content/data/voxel/futa_J-11536_4_2_64_120.npy,120,2-5,後葉,2,1,1,5
...,...,...,...,...,...,...,...,...,...,...
583,futa,J-7607,/content/data/voxel/futa_J-7607_1_0_64_210.npy,210,2-2,前葉,2,1,3,1
584,futa,J-7607,/content/data/voxel/futa_J-7607_1_0_64_240.npy,240,2-2,前葉,2,1,3,1
585,futa,J-7607,/content/data/voxel/futa_J-7607_1_0_64_270.npy,270,2-2,前葉,2,1,3,1
586,futa,J-7607,/content/data/voxel/futa_J-7607_1_0_64_300.npy,300,2-2,前葉,2,1,3,5


## 勾配計算
  category1が型式，category2が年代<br>

GPUで実行すると下記のエラーが出ます．元モデルのBatchNormalization層を削除すれば
GPUでも動きますが，あまり時間は掛からないためCPU実行でも問題ありません．<br>
 UnimplementedError: A deterministic GPU implementation of fused batch-norm backprop, when training is disabled, is not currently available. [Op:FusedBatchNormGradV3]


### クラスタIDの勾配計算
　・マルチラベルで勾配計算する場合，見たいラベルを0，見たいラベル以外を予測値として定義し，Loss関数にマイナスを掛けます．


In [ ]:

for pseudo_df_index in range(0,len(pseudo_df),12):
  
  fname_label = str(pseudo_df.loc[pseudo_df_index,'蓋・身'])+'_'+str(pseudo_df.loc[pseudo_df_index,'列品番号'])
  print(fname_label)

  io_img = np.load(pseudo_df.loc[pseudo_df_index,'voxel_path'])

  io_img = np.expand_dims(io_img, axis=0)
  io_img = tf.cast(io_img, tf.float32)


  test1 =tf.cast(list(pseudo_df.loc[pseudo_df_index, ['クラスタ-型式','クラスタ-時代']]), tf.float32)
  #ワンホットラベルへ変換して    
  one_hot_test1 = np_utils.to_categorical(test1, num_classes=NUM_CLASSES)
  test1 = np.array(one_hot_test1)


  cce = tf.keras.losses.CategoricalCrossentropy()

  #category1：型式の勾配計算
  with tf.GradientTape() as tape1:
    conv_outputs, predictions = grad_model(io_img)
    loss1 = cce(np.zeros(NUM_CLASSES).reshape(1,NUM_CLASSES), predictions[0]) 
    loss2 = cce(test1[1].reshape(1,NUM_CLASSES), predictions[1]) 
    grads1 = tape1.gradient([loss1,loss2], conv_outputs) * -1


  #category2：時代の勾配計算
  with tf.GradientTape() as tape2:
    conv_outputs, predictions = grad_model(io_img)
    loss1 = cce(test1[0].reshape(1,NUM_CLASSES), predictions[0]) 
    loss2 = cce(np.zeros(NUM_CLASSES).reshape(1,NUM_CLASSES), predictions[1])
    grads2 = tape2.gradient([loss1,loss2], conv_outputs) * -1


  output = conv_outputs[0]


  weights1 = tf.reduce_mean(grads1[0], axis=(0,1,2))
  cam1 = np.zeros(output.shape[:3], dtype=np.float32)

  weights2 = tf.reduce_mean(grads2[0], axis=(0,1,2))
  cam2 = np.zeros(output.shape[:3], dtype=np.float32)

  #cam1：型式
  for i, w in enumerate(weights1):
      cam1 += w * output[:, :, :, i]

  #cam2：時代
  for i, w in enumerate(weights2):
      cam2 += w * output[:, :, :, i]


  capi1 = cam1.numpy().repeat(IMAGE_SIZE/len(cam1), axis=0).repeat(IMAGE_SIZE/len(cam1), axis=1).repeat(IMAGE_SIZE/len(cam1), axis=2)
  np.save("{}_{}_cluster1-type.npy".format(fname_label,pseudo_df.loc[pseudo_df_index,'回転角度']), capi1)

  capi2 = cam2.numpy().repeat(IMAGE_SIZE/len(cam2), axis=0).repeat(IMAGE_SIZE/len(cam2), axis=1).repeat(IMAGE_SIZE/len(cam2), axis=2)
  np.save("{}_{}_cluster2-age.npy".format(fname_label,pseudo_df.loc[pseudo_df_index,'回転角度']), capi2)

  gc.collect()

futa_J-11536
futa_J-11563
futa_J-13305-1
futa_J-13305-2
futa_J-13305-3
futa_J-13305-4
futa_J-13305-5
futa_J-13305-6
futa_J-134
futa_J-139
futa_J-147
futa_J-14876
futa_J-20215
futa_J-20295
futa_J-20297
futa_J-22262
futa_J-22896-10
futa_J-22896-11
futa_J-22896-1
futa_J-22896-2
futa_J-22896-3
futa_J-22896-4
futa_J-22896-5
futa_J-22896-6
futa_J-22896-7
futa_J-22896-8
futa_J-22896-9
futa_J-23049
futa_J-23050
futa_J-23682
futa_J-3457
futa_J-3522
futa_J-3584
futa_J-3589
futa_J-3634
futa_J-3651
futa_J-3667
futa_J-4081
futa_J-4120
futa_J-6564
futa_J-6568
futa_J-6569
futa_J-6570
futa_J-6572
futa_J-6573
futa_J-6574
futa_J-6575
futa_J-7598
futa_J-7607


In [ ]:
gc.collect()

8635

# 3D可視化
  * 須恵器が存在するvoxelのみプロットします．
  * カメラ角度は，上，下，左，右，前，後，からの6つです．

In [ ]:
#クラスタIDの勾配を可視化
cam_cat = 'cluster1-type' #ClusterID-型式
#cam_cat = 'cluster2-age' #ClusterID-時代

In [ ]:
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

def plot_voxel(matrix, capi_matrix, cam_cat, \
               pseudo_series, \
               filename="plot_test", matrix_size=32, points_size=15, \
               camera_angle_list = ["from_above","from_below","from_maxZaxis_side",\
                                    "from_minZaxis_side","from_maxXaxis_side","from_minXaxis_side"]):
  
    fig = plt.figure(figsize=(18, 8))

    ax_1 = fig.add_subplot(131, projection='3d', aspect='auto')

    facecolors_1 = matrix.transpose(2,0,1,3)
    facecolors_1 = facecolors_1 / 255
    filled_1 = facecolors_1[:,:,:,-1] != 0
    x, y, z = np.indices(np.array(filled_1.shape) + 1)
    ax_1.voxels(x, y, z, filled_1, facecolors=facecolors_1, shade=False)
    ax_1.set_title("voxel".format(matrix_size, matrix_size, matrix_size), fontsize=12)
    ax_1.set_xlabel("z")
    ax_1.set_ylabel("x")
    ax_1.set_zlabel("y")
    ax_1.set_box_aspect((1,1,1))



    matrix_2 = matrix.copy()
    ax_2 = fig.add_subplot(132, projection='3d', aspect='auto')

    
    # 0をthresholdに
    threshold = 0
    
    #描画用に0-1へ正規化
    norm_capi_matrix = min_max(capi_matrix, axis=(0,1,2))    

    matrix_2[:,:,:,0] = np.where(((matrix[:,:,:,3] > 1)&(capi_matrix < threshold)), 255, matrix[:,:,:,0])
    matrix_2[:,:,:,1] = np.where(((matrix[:,:,:,3] > 1)&(capi_matrix < threshold)), 255, matrix[:,:,:,1])
    matrix_2[:,:,:,2] = np.where(((matrix[:,:,:,3] > 1)&(capi_matrix < threshold)), 255, matrix[:,:,:,2])

    facecolors_2 = matrix_2.transpose(2,0,1,3)
    facecolors_2 = facecolors_2 / 255
    filled_2 = facecolors_2[:,:,:,-1] != 0
    #x, y, z = np.indices(np.array(filled_2.shape) + 1)

    ax_2.voxels(x, y, z, filled_2, facecolors=facecolors_2, shade=False)
    
    ax_2.set_title("Grad-CAMによる勾配値が{}以上であるvoxel".format(threshold), fontsize=12)
    ax_2.set_xlabel("z")
    ax_2.set_ylabel("x")
    ax_2.set_zlabel("y")

    ax_2.set_box_aspect((1,1,1))


    ax_3 = fig.add_subplot(133, projection='3d', aspect='auto')

    facecolors_3 = matrix.copy()

    norm = Normalize(vmin=np.min(capi_matrix), vmax=np.max(capi_matrix))
    jet_colors = plt.cm.jet(norm(capi_matrix))
    facecolors_3[:,:,:,0] = np.where((matrix[:,:,:,3] > 0), jet_colors[:,:,:,0]*255, facecolors_3[:,:,:,0])
    facecolors_3[:,:,:,1] = np.where((matrix[:,:,:,3] > 0), jet_colors[:,:,:,1]*255, facecolors_3[:,:,:,1])
    facecolors_3[:,:,:,2] = np.where((matrix[:,:,:,3] > 0), jet_colors[:,:,:,2]*255, facecolors_3[:,:,:,2])

    facecolors_3 = facecolors_3.transpose(2,0,1,3)
    facecolors_3 = facecolors_3 / 255

    filled_3 = facecolors_3[:,:,:,-1] != 0
    x, y, z = np.indices(np.array(filled_3.shape) + 1)

    m = cm.ScalarMappable(cmap=plt.cm.jet, norm=norm)
    m.set_array([])
   
    ax_3.voxels(x, y, z, filled_3, facecolors=facecolors_3, shade=False)
    ax_3.set_title("Grad-CAM\n（須恵器が存在するのvoxelのみ）", fontsize=12)
    ax_3.set_xlabel("z")
    ax_3.set_ylabel("x")
    ax_3.set_zlabel("y")

    ax_3.set_box_aspect((1,1,1))

    for camera in camera_angle_list:

      if camera == 'from_above':
        ax_1.view_init(elev=90, azim=0)
        ax_2.view_init(elev=90, azim=0)
        ax_3.view_init(elev=90, azim=0)
      elif camera == 'from_below':
        ax_1.view_init(elev=-90, azim=0)
        ax_2.view_init(elev=-90, azim=0)
        ax_3.view_init(elev=-90, azim=0)
      elif camera == 'from_maxZaxis_side':
        ax_1.view_init(elev=0, azim=0)
        ax_2.view_init(elev=0, azim=0)
        ax_3.view_init(elev=0, azim=0)
      elif camera == 'from_minZaxis_side':
        ax_1.view_init(elev=0, azim=180)
        ax_2.view_init(elev=0, azim=180)
        ax_3.view_init(elev=0, azim=180)
      elif camera == 'from_maxXaxis_side':
        ax_1.view_init(elev=0, azim=90)
        ax_2.view_init(elev=0, azim=90)
        ax_3.view_init(elev=0, azim=90)
      elif camera == 'from_minXaxis_side':
        ax_1.view_init(elev=0, azim=-90)
        ax_2.view_init(elev=0, azim=-90)
        ax_3.view_init(elev=0, azim=-90) 

      
      plt.suptitle('{}-{}  (voxel-size:{})    Camera-angle:{}\n '\
                   'true-型式:{},        true-時代:{},    '\
                   '疑似-型式:{},    疑似-時代:{}，     '\
                   'クラスタ-型式:{},    クラスタ-時代:{} '.format(\
                                       filename, cam_cat, matrix_size, camera, \
                                       pseudo_series['正解ラベル-型式'],
                                       pseudo_series['正解ラベル-時代'],
                                       pseudo_series['疑似ラベル-型式'],
                                       pseudo_series['疑似ラベル-時代'],
                                       pseudo_series['クラスタ-型式'],
                                       pseudo_series['クラスタ-時代'],
                                        ), fontsize=18)
    

      fig.subplots_adjust(left=0.05, right=0.92, bottom=0.05, top=0.90, wspace=0.02, hspace=0.1)
      axpos = ax_3.get_position()
      cax = fig.add_axes([0.93, axpos.y0, 0.02, axpos.height])
      cbar = fig.colorbar(m,  cax=cax, orientation='vertical')    

      plt.savefig("{}_{}_{}.png".format(filename, cam_cat, camera))
    #plt.show()
    plt.clf()
    plt.close()
    gc.collect()

In [ ]:
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if val in v[0]]

camera_angle_list = ['from_above',
                     'from_below',
                     'from_maxZaxis_side','from_minZaxis_side','from_maxXaxis_side','from_minXaxis_side'
                     ]

for pseudo_df_index in range(0,len(pseudo_df),12): 
  print(pseudo_df_index)

  fname_label = str(pseudo_df.loc[pseudo_df_index,'蓋・身'])+'_'+str(pseudo_df.loc[pseudo_df_index,'列品番号'])
  capidata_path = "{}_{}_{}.npy".format(fname_label,pseudo_df.loc[pseudo_df_index,'回転角度'],cam_cat)
  data_name = "{}_{}".format(fname_label,pseudo_df.loc[pseudo_df_index,'回転角度'])
  print(data_name)
  voxel_path = pseudo_df.loc[pseudo_df_index,'voxel_path']

  MT_SIZE = 64
  points_size = 32
  print(voxel_path)
  print(capidata_path)
  plot_voxel(np.load(voxel_path), \
            capi_matrix=np.load(capidata_path), cam_cat=cam_cat,\
            pseudo_series=pseudo_df.loc[pseudo_df_index,:],\
            filename=data_name, matrix_size=MT_SIZE, points_size=points_size, \
             camera_angle_list =camera_angle_list)